In [1]:
import os

folders = ["raw", "processed", "output"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Folders created: raw/, processed/, output/")


Folders created: raw/, processed/, output/


In [3]:
import shutil

shutil.move("/content/retail_sales_dataset.csv", "raw/retail_sales.csv")


'raw/retail_sales.csv'

In [4]:
import pandas as pd

df = pd.read_csv("raw/retail_sales.csv")
df.head()


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [5]:
print(df.shape)
df.info()


(1000, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    1000 non-null   int64 
 1   Date              1000 non-null   object
 2   Customer ID       1000 non-null   object
 3   Gender            1000 non-null   object
 4   Age               1000 non-null   int64 
 5   Product Category  1000 non-null   object
 6   Quantity          1000 non-null   int64 
 7   Price per Unit    1000 non-null   int64 
 8   Total Amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


In [6]:
df.isnull().sum()


,0
Transaction ID,0
Date,0
Customer ID,0
Gender,0
Age,0
Product Category,0
Quantity,0
Price per Unit,0
Total Amount,0


In [7]:
df = df.dropna()


In [8]:
df = df.drop_duplicates()


In [9]:
df.columns = df.columns.str.lower().str.replace(" ", "_")


In [11]:
df['date'] = pd.to_datetime(df['date'])
df['total_amount'] = df['total_amount'].astype(float)

In [13]:
total_revenue = df['total_amount'].sum()
df['revenue_contribution_pct'] = (df['total_amount'] / total_revenue) * 100


In [15]:
df['high_value_customer'] = df['total_amount'].apply(lambda x: 1 if x > 1000 else 0)

In [17]:
customers = df[['customer_id', 'gender', 'age']].drop_duplicates()
orders = df[['transaction_id', 'date', 'customer_id', 'quantity', 'price_per_unit', 'total_amount']].rename(columns={'transaction_id': 'order_id', 'date': 'order_date', 'total_amount': 'sales'})
products = df[['product_category']].drop_duplicates().rename(columns={'product_category': 'category'})

In [18]:
customers.to_csv("output/customers.csv", index=False)
orders.to_csv("output/orders.csv", index=False)
products.to_csv("output/products.csv", index=False)

print("CSV files saved in output folder")


CSV files saved in output folder


In [19]:
import sqlite3

conn = sqlite3.connect("output/database.sqlite")

customers.to_sql("customers", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)

conn.close()
print("Data loaded into SQLite database")


Data loaded into SQLite database


In [20]:
print("Raw Data Rows:", df.shape[0])
print("Customers:", customers.shape[0])
print("Orders:", orders.shape[0])
print("Products:", products.shape[0])


Raw Data Rows: 1000
Customers: 1000
Orders: 1000
Products: 3
